# 얼굴 인식을 통한 사진 분류
    최초 작성일 : 20/03/01
    작성자 : 양희승

    작성내용 : 기본 사이즈 변경
               얼굴 인식으로 사진 분류
               얼굴 크롭해서 저장
               
    수정내용
        20/03/02
            - 사진의 갯수가 많아질수록 느려지므로 
                멀티스레드
                멀티프로세싱 테스트
            - 사진 화질 개선
            - 저장 포멧 변경  JPG -> png 

In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from threading import Thread
import multiprocessing
import timeit

## 함수

In [10]:
def resizer(img):
    h, w, c = img.shape
    
    if h == 200 :
        re_img = img
    else :
        tmp = 200/h
        re_img = cv.resize(img, dsize=(0, 0), fx= tmp, fy= tmp, interpolation=cv.INTER_LINEAR)
        

    return re_img

In [11]:
def photo_sorter(origin_img, file_name) :
    h, w, c = origin_img.shape

    if (h > 2000) or (w > 2000) :
        if h < w :
            tmp = round(2000/w, 3)
        elif h > w :
            tmp = round(2000/h, 3)
        else :
            tmp = round(2000/h, 3)

        re_img = cv.resize(origin_img, dsize=(0, 0), fx= tmp, fy= tmp, interpolation=cv.INTER_LINEAR)
    else :
        re_img = origin_img
    
    gray = cv.cvtColor(re_img, cv.COLOR_BGR2GRAY) 
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    
    if len(faces) > 0 :
        for i in range(len(faces)):
            x, y, w, h = faces[i]
            crop_img = re_img[y:y+h, x:x+w]
            re_crop_img = resizer(crop_img)
            cv.imwrite("crop/"+file_name+"_"+str(i)+".png", re_crop_img)
    else :
        pass

In [12]:
def get_count(num, p=4) :
    lists = []
    allocate = int(num/p)
    for n in range(p) :
        lists.append(allocate)
        
    lists[p-1] += num%p
    print("프로세스 할당량 :", lists)
    return lists

In [15]:
def work(a, b):
    for i in range(a, b, 1) :
        file_name = "img ("+str(i) + ")"
        img = cv.imread("photo/"+file_name+".JPG")
    #     plt.imshow(img)
    #     plt.show()
        photo_sorter(img, file_name)
            
        if i%100 == 0 :
            print(file_name, "까지 완료")
        elif i%3==0 :
            print("-", end="")
    return 0

#     file_name = "img ("+str(i) + ")"
#     img = cv.imread("photo/"+file_name+".JPG")
#     photo_sorter(img, file_name)

#     if i%100 == 0 :
#         print(file_name, "까지 완료")
#     elif i%3==0 :
#         print("-", end="")
    

## 코딩 실행 테스트용

In [6]:
## 테스트용  
## 코드 잘 되나??  -> 잘되넹
## 화질 좋아졌나?? -> 좋아졌넹 -> 다만 용량이 좀 늘어났다

start = timeit.default_timer()
faceCascade = cv.CascadeClassifier('data/haarcascade_frontface.xml') 
print("완료 뜰 때까지 기다리세요")

work(1, 100)
stop = timeit.default_timer()
end = round(stop - start, 0)
print("총 걸린시간 : ", end,"초")
print("완료")

완료 뜰 때까지 기다리세요
---------------------------------총 걸린시간 :  44.0 초
완료


## 테스트용 (6511장)

    멀티스레드 사용시
    멀티스레드 사용 안할시 
    
    시간 차이 검토

In [ ]:
# 테스트용 !!! 
# 멀티스레드 사용

# 엥? 왜 죽어버리지

start = timeit.default_timer()
print("완료 뜰 때까지 기다리세요")
faceCascade = cv.CascadeClassifier('data/haarcascade_frontface.xml') 


# th1 = Thread(target=work, args=(1, 1626))
# th2 = Thread(target=work, args=(1626, 3251))
# th3 = Thread(target=work, args=(3251, 4876))
# th4 = Thread(target=work, args=(4876, 6512))

# th1 = Thread(target=work, args=(1, 26))
# th2 = Thread(target=work, args=(26, 51))
# th3 = Thread(target=work, args=(51, 76))
# th4 = Thread(target=work, args=(76, 101))

th1 = Thread(target=work, args=(1, 2170))
th2 = Thread(target=work, args=(2170, 4340))
th3 = Thread(target=work, args=(4340, 6511))


th1.start()
th2.start()
th3.start()
# th4.start()

th1.join()
th2.join()
th3.join()
# th4.join()

stop = timeit.default_timer()
end = round(stop - start, 0)
print("총 걸린시간 : ", end,"초")
print("완료")

완료 뜰 때까지 기다리세요


In [16]:
# 멀티 프로세싱   --- 인터프리터에서는 작동이 되지 않는다. 즉 쥬피터에선 사용 불가

# 파이참을 이용할 것

start = timeit.default_timer()
print("완료 뜰 때까지 기다리세요")
faceCascade = cv.CascadeClassifier('data/haarcascade_frontface.xml') 

num = int(6511)
process_num = 3

process = []
start_num = 1
end_num = 1


for count in get_count(num, process_num) :
    end_num += count
    print("실행 범위", start_num, "~", end_num)
    p = multiprocessing.Process(target=work, args=(start_num, end_num))
    start_num += count
    process.append(p)
    p.start()
    
for p in process :
    p.join


stop = timeit.default_timer()
end = round(stop - start, 0)
print("총 걸린시간 : ", end,"초")
print("완료")

완료 뜰 때까지 기다리세요
프로세스 할당량 : [2170, 2170, 2171]
실행 범위 1 ~ 2171
실행 범위 2171 ~ 4341
실행 범위 4341 ~ 6512
총 걸린시간 :  0.0 초
완료


In [25]:
## 테스트용 (멀티스레드 사용 안했을시)

print("완료 뜰 때까지 기다리세요")
faceCascade = cv.CascadeClassifier('data/haarcascade_frontface.xml') 
work(1, 6512)
stop = timeit.default_timer()
end = round(stop - start, 0)
print("총 걸린시간 : ", end,"초")
print("완료")

완료 뜰 때까지 기다리세요
---------------------------------img (100) 까지 완료
---------------------------------img (200) 까지 완료
---------------------------------img (300) 까지 완료
---------------------------------img (400) 까지 완료
---------------------------------img (500) 까지 완료
---------------------------------img (600) 까지 완료
---------------------------------img (700) 까지 완료
---------------------------------img (800) 까지 완료
---------------------------------img (900) 까지 완료
---------------------------------img (1000) 까지 완료
---------------------------------img (1100) 까지 완료
---------------------------------img (1200) 까지 완료
---------------------------------img (1300) 까지 완료
---------------------------------img (1400) 까지 완료
---------------------------------img (1500) 까지 완료
---------------------------------img (1600) 까지 완료
---------------------------------img (1700) 까지 완료
---------------------------------img (1800) 까지 완료
---------------------------------img (1900) 까지 완료
---------------------------------img (2000) 